### Galois Fields of Degree 2

Burton Rosenberg

November 5, 2021



In [11]:

class QuadInt:
    
    def __init__(self,p,d):
        assert d%p!=0
        assert not self.qr(d,p)

        # p should be a prime but we only hope so
        assert p>2

        self.p = p
        self.d = d%p

    def qr(self,a,p):
        """
        check if a is quadradic residue mod p
        """
        return a**((p-1)//2)%p==1
        
    def get_mod(self):
        return self.p
    
    def get_des(self):
        return self.d

    def canonical(self,a):
        return (a[0]%self.p,a[1]%self.p)

    def zero(self):
        return (0,0)
    
    def one(self):
        return (1,0)

    def zero_p(self,a):
        return self.canonical(a)==self.zero()
    
    def one_p(self,a):
        return self.canonical(a)==self.one()
    
    def equal_p(self,a,b):
        return self.canonical(a)==self.canonical(b)

    def conj(self,a):
        return (a[0],-a[1]%self.p)

    def minus(self,a):
        return (-a[0]%self.p,-a[1]%self.p)

    def add(self,a,b):
        c = (a[0]+b[0]) %self.p
        d = (a[1]+b[1]) %self.p
        return (c,d)
    
    def mult(self,a,b):
        c = (a[0]*b[0]+self.d*a[1]*b[1]) %self.p
        d = (a[0]*b[1]+a[1]*b[0]) %self.p
        return (c,d)
    
    def exp(self,a,n):
        if n==0:
            return (1,0)
        if n==1:
            return a
        if n%2==0:
            c = self.exp(a,n//2)
            return self.mult(c,c)
        c = self.exp(a,n-1)
        return self.mult(a,c)
 
    def extended_gcd(self,a,b):
        """
        extended GCD algorithm. recursive. returns (d,s,t) 
        where d = s*a+t*b and d = gcd(a,b)
        """
        assert(
            a>=0 and b>=0 )
        if b==0:
            return (a,1,0)
        (q,r) = divmod(a,b)
        (d,s,t) = self.extended_gcd(b,r)
        # gcd(a, b) == gcd(b, r) == s*b + t*r == s*b + t*(a - q*b)
        return (d,t,s-q*t)

    def invert_base(self,c):
        assert c[1]==0
        a = c[0]
        (d,t,s) = self.extended_gcd(a,self.p)
        assert 1==d
        return (t%self.p,0)
    
    def invert(self,a):
        if self.zero_p(a):
            return (0,0)

        c = (a[0],-a[1])
        d = self.mult(a,c)
        d_inv = self.invert_base(d)
        return self.mult(c,d_inv)
    
    def orbit_length(self,g):
        if self.zero_p(g):
            return 0
        gg = g
        i  = 1
        while not self.one_p(gg):
            gg = self.mult(gg,g)
            i += 1
        return i

    def gen_p(self,g):
        pp = self.p*self.p-1
        return self.orbit_length(g)==pp

    def next(self,a):
        if a[1]==(self.p-1):
            return ((a[0]+1)%self.p,0)
        return (a[0],(a[1]+1)%self.p)
        
    def str(self,c):
        return f'{c[0]} + {c[1]}\u221A{self.d}'
    
    def __str__(self):
        return f'Z/{self.p}Z[\u221A{self.d}]'


In [12]:
qi = QuadInt(11,7)
print(f'field is: {qi}')

def test_inv(qi):
    p = qi.get_mod()
    a = (0,1)
    while not qi.zero_p(a):
        b = qi.invert(a)
        c = qi.mult(a,b)
        assert qi.one_p(c), f'{a}'
        a = qi.next(a)
    return True

def test_fermat(qi):
    p = qi.get_mod()
    p_x = p*p-1
    a = (0,1)
    while not qi.zero_p(a):
        assert qi.one_p(qi.exp(a,p_x))
        a = qi.next(a)
    return True

def test_squares(qi):
    p = qi.get_mod()
    a = (0,1)
    sqs = []
    i = 0
    while not qi.zero_p(a):
        b1 = qi.mult(a,a)
        a_minus = qi.minus(a)
        b2 = qi.mult(a_minus,a_minus)
        assert qi.equal_p(b1,b2)
        if  b1 not in sqs:
            sqs += [b1]
        i += 1
        a = qi.next(a)
    assert len(sqs)==((p*p-1)//2)
    return True
        
if test_inv(qi):
    print(f'*** passed test_inv')
if test_fermat(qi):
    print(f'*** passesd test_fermat')
if test_squares(qi):
    print(f'*** passesd test_squares')

field is: Z/11Z[√7]
*** passed test_inv
*** passesd test_fermat
*** passesd test_squares


In [14]:

def find_gen(qi):
    a = (0,1)
    gens = []
    while not qi.zero_p(a):
        if qi.gen_p(a):
            gens += [a]
        a = qi.next(a)
    return gens

i = 0 
for g in find_gen(qi):
    i += 1
    print(qi.str(g))
print(f'are the {i} generators of {qi}')

1 + 2√7
1 + 9√7
2 + 3√7
2 + 4√7
2 + 7√7
2 + 8√7
3 + 1√7
3 + 10√7
4 + 3√7
4 + 5√7
4 + 6√7
4 + 8√7
5 + 1√7
5 + 2√7
5 + 9√7
5 + 10√7
6 + 1√7
6 + 2√7
6 + 9√7
6 + 10√7
7 + 3√7
7 + 5√7
7 + 6√7
7 + 8√7
8 + 1√7
8 + 10√7
9 + 3√7
9 + 4√7
9 + 7√7
9 + 8√7
10 + 2√7
10 + 9√7
are the 32 generators of Z/11Z[√7]
